[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/025_mlp_framework/mlp_framework.ipynb)

## MLP y Capas

In [65]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import random

In [66]:
# clase MLP, la red neuronal
class MLP:
    # Inicializador donde se pasa la lista de capas y las guarda
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, x):
        # calculamos la salida del modelo aplicando
        # cada capa de manera secuencial
        # Se itera por la lista de capas y a cada capa se pasara los inputs
        # Donde los outputs seran los inputs de la siguiente capa, los ultimos outputs de la ultima capa son las salidas del perceptron multicapa
        for layer in self.layers:
            x = layer(x)
        return x

Ahora definimos las diferentes capas que necesitamos. En primer lugar tendremos una clase base que contendrá los elementos y funcionalidad común de cada capa. Esta clase contendrá una lista con los parámetros entrenables de la capa y sus gradientes.

In [67]:
# Implementacion de las capas esta es un tipo de clase padre que heredara a sus hijos
class Layer():
    def __init__(self):
        # Lista de parametros que guarda parametros para ser actualizados por el optimizador
        self.params = []
        # Lista con los gradientes que luego seran aplicados a lo sparametros
        self.grads = []

    # la funcion recibe inputs y devuelve los inputs como si fueran outputs
    def __call__(self, x):
        # por defecto, devolver los inputs
        # cada capa hará algo diferente aquí
        return x

    # Funcion que recibe gradientes de las capas y calculara sus propios gradientes y devolvera los gradientes que tiene que pasar a las capas de abajo
    def backward(self, grad):
        # cada capa, calculará sus gradientes
        # y los devolverá para las capas siguientes
        return grad

    # funcion que actualizara los parametros segun lo que le diga el optimizador
    def update(self, params):
        return

Ahora podemos definir las diferentes capas que utilizaremos. Hasta ahora sólo hemos visto la capa lineal y diferentes funciones de activación.

In [68]:
# El perceptron
class Linear(Layer):
    def __init__(self, d_in, d_out):
        # pesos de la capa, es una matriz con dimensiones de entrada y de salida con los pesos
        self.w = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(d_in+d_out)),
                                  size=(d_in, d_out))
        # bias
        self.b = np.zeros(d_out)

    # Funcion que retorna la multiplicacion de los pesos y los inputs y se sumara el bias
    def __call__(self, x):
        # Recibe los inputs y se guardan para usar en la funcion backward
        self.x = x
        self.params = [self.w, self.b]
        # salida del perceptron
        return np.dot(x, self.w) + self.b

    def backward(self, grad_output):
        # 1ro calcula el gradiente que tiene que pasar a la siguiente capa (BACKPROP)
        # donde multiplica la matriz de pesos por el gradiente de la capa anterior. es lo que retorna la funcion para ser utilizado en la capa siguiente
        grad = np.dot(grad_output, self.w.T)
        # gradientes con respecto a los pesos, producto d els inputs por el gradiente de la capa
        self.grad_w = np.dot(self.x.T, grad_output)
        # gradientes para actualizar pesos
        self.grad_b = grad_output.mean(axis=0)*self.x.shape[0]
        self.grads = [self.grad_w, self.grad_b]
        return grad
  # Le va cambiando valores d elos pesos y el bies en base al optimizador
    def update(self, params):
        self.w, self.b = params

In [69]:
# funciones de activacion
class ReLU(Layer):
    # Recibe inputs y las almacena, devuelve el valor maximo enre los inputs y 0
    def __call__(self, x):
        self.x = x
        return np.maximum(0, x)
    # Se calcula la derivada de la relu y se multiplica por gradiente de la capa siguiente
    def backward(self, grad_output):
        grad = self.x > 0
        return grad_output*grad

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)


## Optimizador

De momento solo conocemos un algoritmo de optimización, el `descenso por gradiente`. En este algoritmo, iteraremos por todas las capas del `MLP` actualizando los parámetros.

In [70]:
# Descenso por el gradiente
class SGD():
    # Le pasamos la red y el learning rate
    def __init__(self, net, lr):
        self.net = net
        self.lr = lr

    # itera por todas las capas de la red neuronal
    def update(self):
        for layer in self.net.layers:
            # a capa capa se le llama a su propia funcion update
            layer.update([
                # Regla de actualizacion
                params - self.lr*grads
                # iteramos los parametros y gradientes de cada capa
                for params, grads in zip(layer.params, layer.grads)
            ])

In [71]:
# Funciones de perdida

# Clase padre Loss
class Loss():
    def __init__(self, net):
        self.net = net

    def backward(self):
        # derivada de la loss function con respecto
        # a la salida del MLP
        grad = self.grad_loss()
        # Se hace todo el proceso de backpropagation
        # Se itera todas las capas pero desde la ultima hasta la primera
        for layer in reversed(self.net.layers):
            grad = layer.backward(grad)

class CrossEntropy(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target
        logits = output[np.arange(len(output)), target.astype(int)]
        loss = - logits + np.log(np.sum(np.exp(output), axis=-1))
        loss = loss.mean()
        return loss

    def grad_loss(self):
        answers = np.zeros_like(self.output)
        answers[np.arange(len(self.output)), self.target.astype(int)] = 1
        return (- answers + softmax(self.output)) / self.output.shape[0]

In [72]:
data = pd.read_csv('train.csv')
print(data)

        Unnamed: 0      id  Gender      Customer Type  Age   Type of Travel  \
0                0   70172    Male     Loyal Customer   13  Personal Travel   
1                1    5047    Male  disloyal Customer   25  Business travel   
2                2  110028  Female     Loyal Customer   26  Business travel   
3                3   24026  Female     Loyal Customer   25  Business travel   
4                4  119299    Male     Loyal Customer   61  Business travel   
...            ...     ...     ...                ...  ...              ...   
103899      103899   94171  Female  disloyal Customer   23  Business travel   
103900      103900   73097    Male     Loyal Customer   49  Business travel   
103901      103901   68825    Male  disloyal Customer   30  Business travel   
103902      103902   54173  Female  disloyal Customer   22  Business travel   
103903      103903   62567    Male     Loyal Customer   27  Business travel   

           Class  Flight Distance  Inflight wifi se

In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  object 
 3   Customer Type                      103904 non-null  object 
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  object 
 6   Class                              103904 non-null  object 
 7   Flight Distance                    103904 non-null  int64  
 8   Inflight wifi service              103904 non-null  int64  
 9   Departure/Arrival time convenient  103904 non-null  int64  
 10  Ease of Online booking             103904 non-null  int64  
 11  Gate location                      1039

In [75]:
columnas_categoricas = data.select_dtypes(include=['object']).columns
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])

print(data)

        Unnamed: 0      id  Gender  Customer Type  Age  Type of Travel  Class  \
0                0   70172       1              0   13               1      2   
1                1    5047       1              1   25               0      0   
2                2  110028       0              0   26               0      0   
3                3   24026       0              0   25               0      0   
4                4  119299       1              0   61               0      0   
...            ...     ...     ...            ...  ...             ...    ...   
103899      103899   94171       0              1   23               0      1   
103900      103900   73097       1              0   49               0      0   
103901      103901   68825       1              1   30               0      0   
103902      103902   54173       0              1   22               0      1   
103903      103903   62567       1              0   27               0      0   

        Flight Distance  In

In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  int64  
 3   Customer Type                      103904 non-null  int64  
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  int64  
 6   Class                              103904 non-null  int64  
 7   Flight Distance                    103904 non-null  int64  
 8   Inflight wifi service              103904 non-null  int64  
 9   Departure/Arrival time convenient  103904 non-null  int64  
 10  Ease of Online booking             103904 non-null  int64  
 11  Gate location                      1039

In [77]:
#llenar datos vacios
vacios = ['Arrival Delay in Minutes']
media_columnas = data[vacios].mean()
data[vacios] = data[vacios].fillna(media_columnas)

eliminar_columnas = ['Unnamed: 0', 'id']
data = data.drop(eliminar_columnas, axis=1)

In [78]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Gender                             103904 non-null  int64  
 1   Customer Type                      103904 non-null  int64  
 2   Age                                103904 non-null  int64  
 3   Type of Travel                     103904 non-null  int64  
 4   Class                              103904 non-null  int64  
 5   Flight Distance                    103904 non-null  int64  
 6   Inflight wifi service              103904 non-null  int64  
 7   Departure/Arrival time convenient  103904 non-null  int64  
 8   Ease of Online booking             103904 non-null  int64  
 9   Gate location                      103904 non-null  int64  
 10  Food and drink                     103904 non-null  int64  
 11  Online boarding                    1039

In [79]:
data = data.to_numpy()

In [80]:

X = data[:90000, [0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]]
Y = data[:90000, 4]

# normalización datos
X_mean, X_std = X.mean(axis=0), X.std(axis=0)
X_norm = (X - X_mean) / X_std

X.shape, Y.shape

((100000, 22), (100000,))

In [81]:
D_in, H, D_out = 22, 100, 3

# Perceptron multicapa donde se pasa la lista de capas
mlp = MLP([
    Linear(D_in, H),
    ReLU(),
    Linear(H, H),
    ReLU(),
    Linear(H, H),
    ReLU(),
    Linear(H, H),
    ReLU(),
    Linear(H, D_out)
])
# Calcula la loss function y empezar el calculo d elos gradientes, inicializando el backpropagation
loss = CrossEntropy(mlp)

In [87]:
# Definimos el objeto objeto optimizador que actualiza los pesos
optimizer = SGD(mlp, lr=0.001)


# Bucle de entrenamiento
epochs = 200 # cantidad de veces que entrenara todo el dataset
batch_size = 100  # agarra grupos de 100  en cada epoch para ano llenar memorias

batches = len(X) // batch_size
log_each = 10 # cada cuantos epoch imprime   10/200
l = [] # perdida total

# Se itera por unas cuantas epoch y luego se itera por el dataset en batches
for e in range(1,epochs+1):
    _l = [] # perdida por cada epoch
    for b in range(batches):
        x = X_norm[b*batch_size:(b+1)*batch_size]
        y = Y[b*batch_size:(b+1)*batch_size]
        # Se calcula la salida del perceptron pasando las entradas al objeto MLP
        y_pred = mlp(x)
        # Se pasa las predicciones para calcular la funcion de perdida y se ira almacenando en la lista
        _l.append(loss(y_pred, y))
        # Se llama a la funcion backward para iniciar el backpropagation calculando todas las derivadas de la funcion de perdida con respecto a todos los pesos, las capas que sean automaticamente se calcularan todos los gradientes
        loss.backward()
        # El optimizador actualizara los pesos
        optimizer.update()
    l.append(np.mean(_l))
    if not e % log_each:
        print(f'Epoch {e}/{epochs}, Loss: {np.mean(l):.4f}')

# Epoch 500 saltos 50,  Loss:0.1111 200 neuronas  99.83% de relacion

Epoch 10/200, Loss: 0.1341
Epoch 20/200, Loss: 0.1135
Epoch 30/200, Loss: 0.1027
Epoch 40/200, Loss: 0.0955
Epoch 50/200, Loss: 0.0901
Epoch 60/200, Loss: 0.0858
Epoch 70/200, Loss: 0.0822
Epoch 80/200, Loss: 0.0791
Epoch 90/200, Loss: 0.0764
Epoch 100/200, Loss: 0.0741
Epoch 110/200, Loss: 0.0719
Epoch 120/200, Loss: 0.0699
Epoch 130/200, Loss: 0.0681
Epoch 140/200, Loss: 0.0665
Epoch 150/200, Loss: 0.0649
Epoch 160/200, Loss: 0.0635
Epoch 170/200, Loss: 0.0622
Epoch 180/200, Loss: 0.0609
Epoch 190/200, Loss: 0.0597
Epoch 200/200, Loss: 0.0586


In [83]:
#Array de prueba
X_test = X[90000:, :].copy()

# Normaliza los datos de prueba porque los estamos agarrando in normalizar
X_test_norm = (X_test - X_mean) / X_std

# Pasa los datos de prueba a través del modelo para obtener las predicciones
y_pred_test = mlp(X_test_norm)

# Convierte las predicciones a clases elige el maximo de los valores
prediccion = np.argmax(y_pred_test, axis=1)

# verificamos los resultados
print("Predicciones del modelo:")
print(prediccion)
print("Valores reales:")
print(Y[90000:])

# Calcula el porcentaje de relación entre las predicciones y los valores reales
total_ejemplos = len(prediccion)
correlacion = np.sum(prediccion == Y[90000:])
porcentaje = (correlacion / total_ejemplos) * 100


print("Porcentaje de relación:")
print(f"{porcentaje:.2f}%")


Predicciones del modelo:
[1 0 1 ... 0 0 0]
Valores reales:
[2. 1. 1. ... 0. 0. 0.]
Porcentaje de relación:
95.99%


In [90]:
def predict_one():
    #Valor de prueba
    x_index = random.randint(0, 100000)

    X_test = X[x_index, :].copy()

    X_test_norm = (X_test - X_mean) / X_std

    y_pred_test = mlp(X_test_norm)

    prediccion = np.argmax(y_pred_test)

    print("Prediccion del modelo:")
    print(prediccion)
    print("Valor real:")
    print(f"{Y[x_index]:.0f}\n")

In [91]:
for i in range(5):
    predict_one()

Prediccion del modelo:
1
Valor real:
1

Prediccion del modelo:
1
Valor real:
1

Prediccion del modelo:
1
Valor real:
1

Prediccion del modelo:
1
Valor real:
1

Prediccion del modelo:
1
Valor real:
1

